In [1]:
import theano
# theano.config.device = 'gpu'
# theano.config.floatX = 'float32'
%matplotlib inline
import matplotlib.pyplot as plt
import glob
import numpy as np
from collections import defaultdict


num_class = 256

Using gpu device 0: GeForce GTX TITAN X (CNMeM is enabled with initial size: 95.0% of memory, cuDNN 5005)


In [2]:
import keras
from keras.applications import VGG16
from keras.models import Model
from keras.layers import Dense
from keras.optimizers import SGD
from keras.initializations import normal


def getModel( output_dim ):
    ''' 
        * output_dim: the number of classes (int)
        * return: compiled model (keras.engine.training.Model)
    '''
    vgg_model = VGG16( weights='imagenet', include_top=True )
    vgg_out = vgg_model.layers[-2].output #Last FC layer's output  
    softmax_layer = Dense(output_dim, activation='softmax')(vgg_out) #Create softmax layer taking input as vgg_out
    #Create new transfer learning model
    tl_model = Model( input=vgg_model.input, output=softmax_layer )

    #Freeze all layers of VGG16 and Compile the model
    for layer in vgg_model.layers:
        layer.trainable = False
    #Confirm the model is appropriate
    return tl_model

model = getModel(num_class)
# sgd = SGD(lr=1e-3, decay=1e-4, momentum=0.9, nesterov=True)
# model.compile(optimizer='rmsprop',
#               loss='categorical_crossentropy',
#               metrics=['accuracy'])

Using Theano backend.


In [3]:
def list_gen():
    root_dir = '../256_ObjectCategories/'
    image_dirs = glob.glob(root_dir + '*')
    d = defaultdict(list)
    for idir in image_dirs:
        all_images = glob.glob(idir + '/*.jpg')
        label_id = int(idir.split('/')[-1].split('.')[0])-1
        if label_id > 255:
            continue
        np.random.shuffle(all_images)
        train_paths, val_paths, test_paths = all_images[:len(all_images)/10*8], all_images[len(all_images)/10*8:len(all_images)/10*9], all_images[len(all_images)/10*9:]
        train_labels, val_labels, test_labels = [label_id]*(len(all_images)/10*8), [label_id]*(len(all_images)/10), [label_id]*(len(all_images)/10)
        
        d['train_images'].append(train_paths)
        d['val_images'].append(val_paths)
        d['test_images'].append(test_paths)
        d['train_labels'].append(train_labels)
        d['val_labels'].append(val_labels)
        d['test_labels'].append(test_labels)
    return d

all_d = list_gen()

In [4]:
from keras.preprocessing.image import load_img, img_to_array
from keras.utils import np_utils
from keras.applications.imagenet_utils import preprocess_input

def load_data(image_paths, labels, crop_size):
    X = np.zeros((len(image_paths), crop_size, crop_size, 3))
    for i,path in enumerate(image_paths):
        X[i, :] = img_to_array(load_img(path, target_size=(crop_size, crop_size)))
    y = np_utils.to_categorical(labels)
    return X, y

In [5]:
crop_size = 224
all_per_class = 16
train_paths = []
val_paths = []
test_paths = []
train_labels = []
val_labels = []
test_labels = []
for i in xrange(num_class):
    train_paths += (all_d['train_images'][i][:all_per_class])
    val_paths += (all_d['val_images'][i][:all_per_class])
    test_paths += (all_d['test_images'][i][:all_per_class])
    train_labels += (all_d['train_labels'][i][:all_per_class])
    val_labels += (all_d['val_labels'][i][:all_per_class])
    test_labels += (all_d['test_labels'][i][:all_per_class])

X_train, y_train = load_data(train_paths, train_labels, crop_size)
X_val, y_val = load_data(val_paths, val_labels, crop_size)
X_test, y_test = load_data(test_paths, test_labels, crop_size)

In [6]:
X_train = preprocess_input(X_train)
X_val = preprocess_input(X_val)
X_test = preprocess_input(X_test)

In [7]:
from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(
#     featurewise_center=True,
#     featurewise_std_normalization=True,
#     rotation_range=20,
#     width_shift_range=0.2,
#     height_shift_range=0.2,
    rescale=1./128)

datagen.fit(X_train)

In [9]:
num_epoch = 50
batch_size = 64

for sample_per_class in [2,4,8,16]:
    train = X_train.reshape((num_class, -1, crop_size, crop_size, 3))
    train = train[:, :sample_per_class].reshape((-1, crop_size, crop_size, 3))
    label = y_train.reshape((num_class, -1, num_class))
    label = label[:, :sample_per_class].reshape((-1, num_class))
    rmsmodel = getModel(num_class)
    rmsmodel.compile(optimizer='rmsprop',
                     loss='categorical_crossentropy',
                     metrics=['accuracy'])
    rmsmodel.fit_generator(datagen.flow(train, label, batch_size=32),
                           validation_data=datagen.flow(X_val, y_val, batch_size=batch_size),
                           nb_val_samples=200,
                           samples_per_epoch=len(train), nb_epoch=num_epoch)

Epoch 1/50
512/512 [==============================] - 6s - loss: 10.2884 - acc: 0.0020 - val_loss: 7.5286 - val_acc: 0.0039
Epoch 2/50
512/512 [==============================] - 6s - loss: 6.5496 - acc: 0.0137 - val_loss: 6.1474 - val_acc: 0.0078
Epoch 3/50
512/512 [==============================] - 6s - loss: 5.7005 - acc: 0.0215 - val_loss: 5.9528 - val_acc: 0.0430
Epoch 4/50
512/512 [==============================] - 6s - loss: 5.0559 - acc: 0.0391 - val_loss: 5.8071 - val_acc: 0.0391
Epoch 5/50
512/512 [==============================] - 6s - loss: 4.4553 - acc: 0.0723 - val_loss: 5.2556 - val_acc: 0.0938
Epoch 6/50
512/512 [==============================] - 6s - loss: 3.8692 - acc: 0.1270 - val_loss: 5.0214 - val_acc: 0.0781
Epoch 7/50
512/512 [==============================] - 6s - loss: 3.3450 - acc: 0.1992 - val_loss: 4.7153 - val_acc: 0.1016
Epoch 8/50
512/512 [==============================] - 7s - loss: 2.8636 - acc: 0.2676 - val_loss: 5.1866 - val_acc: 0.0820
Epoch 9/50
512/

In [ ]:
train_loss=[[],[],[],[]]
val_loss=[[],[],[],[]]
train_accu=[[],[],[],[]]
val_accu=[[],[],[],[]]
with open('log', 'r') as f:
    for line in f:
        exp = 

In [ ]:
# plotting
# loss vs epoch
plt.figure()
for i,sample_size in enumerate([2,4,8,16]):
    plt.plot(range(num_epoch), train_loss[i], 'b-', label='{} samples /class training'.format(sample_size))
    plt.plot(range(num_epoch), val_loss[i], 'b.', label='{} samples /class validation'.format(sample_size))
plt.title('loss vs # of epochs')
plt.legend('top right')
plt.xlabel('# of epochs')
plt.ylabel('loss')
plt.savefig('images/4a.pdf')
plt.show()

# accu vs epoch
plt.figure()
for i,sample_size in enumerate([2,4,8,16]):
    plt.plot(range(num_epoch), train_accu[i], 'b-', label='{} samples /class training'.format(sample_size))
    plt.plot(range(num_epoch), val_accu[i], 'b.', label='{} samples /class validation'.format(sample_size))
plt.title('accuracy vs # of epochs')
plt.legend('top right')
plt.xlabel('# of epochs')
plt.ylabel('accuracy')
plt.savefig('images/4b.pdf')
plt.show()

# training sample vs accu
plt.figure()
plt.plot([2,4,8,16], [train_accu[i][-1] for i in xrange(4)], label='train')
plt.plot([2,4,8,16], [val_accu[i][-1] for i in xrange(4)], label='val')
plt.xscale('log')
plt.title('accuracy vs # of samples')
plt.legend('top right')
plt.xlabel('# of samples')
plt.ylabel('accuracy')
plt.savefig('images/4c.pdf')
plt.show()


In [26]:
model = getModel(num_class)
# sgd = SGD(lr=1e-3, decay=1e-4, momentum=0.9, nesterov=True)
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit_generator(datagen.flow(X_train, y_train, batch_size=32),
                    samples_per_epoch=len(X_train), nb_epoch=10)

Epoch 1/10
1024/1024 [==============================] - 302s - loss: 8.6288 - acc: 0.0088   
Epoch 2/10
1024/1024 [==============================] - 304s - loss: 6.0655 - acc: 0.0332   
Epoch 3/10
1024/1024 [==============================] - 304s - loss: 5.1832 - acc: 0.0654   
Epoch 4/10
1024/1024 [==============================] - 304s - loss: 4.4927 - acc: 0.1006   
Epoch 5/10
1024/1024 [==============================] - 305s - loss: 3.8219 - acc: 0.1445   
Epoch 6/10
1024/1024 [==============================] - 306s - loss: 3.3476 - acc: 0.2168   
Epoch 7/10
1024/1024 [==============================] - 312s - loss: 2.8463 - acc: 0.2979   
Epoch 8/10
 384/1024 [==========>...................] - ETA: 193s - loss: 2.6298 - acc: 0.3047

KeyboardInterrupt: 